One a high level, logistic regression is a method by which we can calculate the probability that an input belongs to one of the target classes.

We will build the logistic regression model in four phases:
- **inference**: (or predict) which produces a probability distribution over the output classes given a minibatch
- **loss**: which computes the value of the error function (in this case, the cross entropy loss)
- **training**: which is responsible for computing the gradients of the model's parameters and updating the model
- **evaluate**: which will determine the effectiveness of a model

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def inference(x):
    init = tf.constant_initializer(value=0)
    W = tf.get_variable('W', [784, 10], initializer=init)
    b = tf.get_variable('b', [10], initializer=init)
    
    output = tf.nn.softmax(tf.matmul(x, W) + b)
    
    return output
    

In [4]:
def loss(output, y):
    dot_product = - y * tf.log(output) # output: (batch_size, class_size)
    loss = tf.reduce_mean(tf.reduce_sum(dot_product, reduction_indices=1))
    return loss

TensorFlow makes training easy by giving us access to built-in optimizers that produce a special train operation that we can run via a TensorFlow session when we minimize them. Note that when we create the training operation, we also pass in a variable that represents the number of minibatches that has been processed. Each time the training operation is run, this step variable is incremented so that we can keep track of progress.

In [5]:
def train(cost, global_step):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [6]:
def evaluate(output, y):
    correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

We log several summary statistics. For example, we use the *tf.scalar_summary* and *tf.histogram_summary* commands to log the cost for each minibatch, validation error, and the distribution of parameters.

In [7]:
def training(cost, global_step):
    tf.scalar_summary("cost",cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

Every epoch, we run the *tf.merge_all_summaries* in order to collect all summary statistics we've logged and use *tf.train.SummaryWriter* to write the log to disk. We'll also describe how we can visualize these logs with the built-in TensorBoard tool.

In addition to saving summary statistics, we also have the model parameters using the *tf.train.Saver* model saver. 

In [8]:
learning_rate = 0.01
train_epochs = 1000
batch_size = 100
display_step = 1

In [9]:
with tf.Graph().as_default():
    x = tf.placeholder("float", [None, 784])
    y = tf.placeholder("float", [None, 10])
    
    output = inference(x)
    cost = loss(output, y)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = training(cost, global_step)
    eval_op = evaluate(output, y)
    summary_op = tf.merge_all_summaries()
    print('summary_op:', summary_op)
    saver = tf.train.Saver()
    sess = tf.Session()
    summary_writer = tf.train.SummaryWriter("logistic_logs/", graph=sess.graph)
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    
    for epoch in range(train_epochs):
        avg_cost = 0.0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            minibatch_x, minibatch_y = mnist.train.next_batch(batch_size)
            feed_dict = {x : minibatch_x, y : minibatch_y}
            sess.run(train_op, feed_dict = feed_dict)
            minibatch_cost = sess.run(cost, feed_dict=feed_dict)
            avg_cost += minibatch_cost/total_batch
        if epoch % display_step == 0:
            val_feed_dict = {
                x : mnist.validation.images,
                y : mnist.validation.labels
            }
            accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
            print(type(eval_op)) # Tensor
            print(type(accuracy)) # numpy.data
            print("Validation Error:", (1 - accuracy))
            summary_str = sess.run(summary_op, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, sess.run(global_step))
            saver.save(sess, "logistic_logs/model-checkpoint", global_step=global_step)
    
    print("optimization Finished!")
    
    test_feed_dict = {
        x : mnist.test.images,
        y : mnist.test.labels
    }
    
    accuracy = sess.run(eval_op, feed_dict=test_feed_dict)
    print("Test Accuarcy:", accuracy)

summary_op: Tensor("MergeSummary/MergeSummary:0", shape=(), dtype=string)
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validation Error: 0.1507999897
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validation Error: 0.128600001335
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validation Error: 0.120999991894
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validation Error: 0.113399982452
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validation Error: 0.110000014305
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validation Error: 0.107200026512
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validation Error: 0.105400025845
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validation Error: 0.102599978447
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.float32'>
Validat

In [25]:
print(mnist.train.num_examples)
print(int(mnist.train.num_examples/batch_size))

55000
550


** Leveraging TensorBoard to Visualize Computation Graphs and Learning **

Once we set up the logging of summary statistics as described in the previous  section, we are ready to visualize the data we've collected. TensorBoard comes with a visualization tool called TensorBoard which provides an easy-to-use interface for navigating through our summary statistics.

In [1]:
summary_op = tf.merge_all_summaries()
print(summary_op)

NameError: name 'sess' is not defined